In [23]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import torchvision
from torch.utils.data import Subset
from tqdm import tqdm
from torch.distributions.laplace import Laplace
import math


from ntk_utils import gen_h_dis, gen_alpha, gen_z_embed, process_query

# device =
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# Load the pretrained model
model = models.resnet18(pretrained=True)

model = model.to(device)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(img):
    # 1. Load the image with Pillow library
    # img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512).to(device)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        # my_embedding.copy_(o.data)
        my_embedding.copy_(o.data.reshape(o.data.size(1)))
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    # origin_device = t_img.device
    t_img = t_img.to(device)
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()

    return my_embedding

    # my_embedding = my_embedding.to(origin_device)
    # # 8. Return the feature vector
    # return my_embedding.numpy()


ds = torchvision.datasets.CIFAR10(root='./data', train=True, download=False)

def gen_2classes_indices(cls1_name, cls2_name):
    cls1_indices, cls2_indices, other_indices = [], [], []
    cls1_idx, cls2_idx = ds.class_to_idx[cls1_name], ds.class_to_idx[cls2_name]

    for i in range(len(ds)):
        current_class = ds[i][1]
        if current_class == cls1_idx:
            cls1_indices.append(i)
        elif current_class == cls2_idx:
            cls2_indices.append(i)
        else:
            other_indices.append(i)

    return cls1_indices, cls2_indices

def gen_feature_tensor(idx_list):
    img_ts_list = []
    for idx in tqdm(idx_list):
        image, label = ds[idx]
        img_ts = get_vector(image)
        # img_ts = torch.from_numpy(img_np)
        img_ts_list.append(img_ts)

    # concat
    ret_ts = torch.stack(img_ts_list, dim=0)
    return ret_ts


def test_accuracy(test_dataset, gt_label, w_r, x_data, alpha):
    pred = process_query(test_dataset, w_r, x_data, alpha)
    succ_cnt = torch.sum(pred == gt_label)
    nz = pred.shape[0]
    accuracy = succ_cnt / nz
    # print("accuracy", accuracy)
    return accuracy


def cal_k(eps, delta, beta):
    eta = 7e-3
    n = 1e3
    k_bound = (eps * eps * eta * eta) / (8 * math.log(1 / delta) * n * n * beta * beta)
    k = int(math.floor(k_bound))
    return k




/mnt/nvme2/xuhaiyang/softwares/anaconda3/envs/galore/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/nvme2/xuhaiyang/softwares/anaconda3/envs/galore/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:

######### normalization x data start ###########
def calculate_norm(input_data):
    # input_data: n * d
    square_data = input_data * input_data
    norm_data = square_data.sum(dim=1)
    norm_data = torch.sqrt(norm_data)
    return norm_data

def data_normalization(input_data):
    # input data: n * d
    print("data normalized")
    x_norm = calculate_norm(input_data)
    x_norm = x_norm[..., None]
    ball_data = input_data / x_norm

    return ball_data
######### normalization x data end ###########

# there are 5k images for 1 class
train_num = 1000
test_num = 100
label_scale = 1.0

cls1_name = "airplane"
cls2_name = "cat"

cls1_indices, cls2_indices = gen_2classes_indices(cls1_name, cls2_name)

cls1_train_ts = gen_feature_tensor(cls1_indices[:train_num]).to(device)
cls2_train_ts = gen_feature_tensor(cls2_indices[:train_num]).to(device)

cls1_label = torch.full((train_num, ), label_scale, dtype=torch.float32)
cls2_label = torch.full((train_num, ), -1 * label_scale, dtype=torch.float32)

cls1_test_ts = gen_feature_tensor(cls1_indices[-test_num:]).to(device)
cls2_test_ts = gen_feature_tensor(cls2_indices[-test_num:]).to(device)

############# test on NTK Regression start #################

######### normalization start ################
cls1_train_ts = data_normalization(cls1_train_ts)
cls2_train_ts = data_normalization(cls2_train_ts)
cls1_test_ts = data_normalization(cls1_test_ts)
cls2_test_ts = data_normalization(cls2_test_ts)
######### normalization end ################


m = 256
reg_lambda = 10.0

x_data = torch.cat((cls1_train_ts, cls2_train_ts), dim=0).to(device)
y_data = torch.cat((cls1_label, cls2_label), dim=0).to(device)

n, d = x_data.shape

# generate w_r
w_r = torch.randn((m, d), dtype=torch.float32).to(device)

h_dis = gen_h_dis(w_r, x_data)

alpha = gen_alpha(h_dis, reg_lambda, y_data)

# # may scale down alpha here
# alpha = alpha / (n * n)

def gaussain_sampling_on_k(h_dis, y_data, reg_lambda, cls1_test_ts, cls2_test_ts, k=None):
    
    n = h_dis.shape[0]

    test_acc_list = []
    train_acc_list = []

    repeat_time = 10

    for _ in tqdm(range(repeat_time)):
        if k is None:
            wt_h_dis = h_dis
        else:
            # setup gaussian sampler
            gaussian_sampler = torch.distributions.MultivariateNormal(
                loc=torch.zeros(n).to(device), covariance_matrix=h_dis
            )

            # gausian sampling
            wt_h_dis = torch.empty(k, n, n).to(device)

            for i in range(k):
                sample_vec = gaussian_sampler.sample()
                wt_h_dis[i] = sample_vec[..., None] @ sample_vec[None, ...]

            # take mean over dim k
            # n * n
            wt_h_dis = wt_h_dis.mean(dim=0)

        alpha = gen_alpha(wt_h_dis, reg_lambda, y_data)

        alpha = alpha / (n * n)

        cls1_accuracy = test_accuracy(cls1_test_ts, 1, w_r, x_data, alpha)
        cls2_accuracy = test_accuracy(cls2_test_ts, -1, w_r, x_data, alpha)

        cls1_train_acc = test_accuracy(cls1_train_ts, 1, w_r, x_data, alpha)
        cls2_train_acc = test_accuracy(cls2_train_ts, -1, w_r, x_data, alpha)

        cur_test_acc = (cls1_accuracy + cls2_accuracy) / 2
        cur_train_acc = (cls1_train_acc + cls2_train_acc) / 2

        # print(cur_test_acc, cur_train_acc)

        test_acc_list.append((cls1_accuracy + cls2_accuracy) / 2)
        train_acc_list.append((cls1_train_acc + cls2_train_acc) / 2)

    final_test_acc = sum(test_acc_list) / len(test_acc_list)
    final_train_acc = sum(train_acc_list) / len(train_acc_list)

    return final_test_acc, final_train_acc


final_test_acc, final_train_acc = gaussain_sampling_on_k(h_dis, y_data, reg_lambda, cls1_test_ts, cls2_test_ts, None)

# print(exponent)
# print(k
print("test acc", final_test_acc)
print("train acc", final_train_acc)


100%|██████████| 100/100 [00:00<00:00, 203.52it/s]


data normalized
data normalized
data normalized
data normalized


100%|██████████| 10/10 [00:13<00:00,  1.38s/it]

test acc tensor(0.9800, device='cuda:0')
train acc tensor(0.9810, device='cuda:0')


In [17]:
eigen_value = torch.linalg.eigvals(h_dis)
print(min(eigen_value.real))

tensor(0.0075, device='cuda:0')


In [30]:


############# test non-private alpha start #############
# final_test_acc, final_train_acc = gaussain_sampling_on_k(h_dis, y_data, reg_lambda, cls1_test_ts, cls2_test_ts, None)

# print("test acc", final_test_acc)
# print("train acc", final_train_acc)

# raise
########### test non-private alpha end ################


# beta = 0.05
# delta = 1e-3
# eps_exponent_list = [-2.0 + i * 0.2 for i in range(11)]

# eps_exponent_list = [-1.5 + i * 0.15 for i in range(11)]


# # eps_exponent_list = eps_exponent_list[:1]

# print(eps_exponent_list)


beta = 1e-7
delta = 1e-3
# test_k = cal_k(1e-2, delta, beta)

# print(test_k)

# eps_exponent_list = [-1.0 + i * 0.2 for i in range(11)]
# eps_exponent_list = [-0.5 + i * 0.1 for i in range(11)]

eps_exponent_list = [-0.4, -0.3]

# for eps_exponent in eps_exponent_list:
#     eps = 10 ** eps_exponent
#     k = cal_k(eps, delta, beta)

#     print(f"eps exponent {eps_exponent}, k {k}")

# raise

# raise


draw_test_acc_list = []
draw_train_acc_list = []

# for k in k_list:
for eps_exponent in eps_exponent_list:
    eps = 10 ** eps_exponent
    k = cal_k(eps, delta, beta)

    if k > 2549:
        k = 2500

    print("-" * 50)

    print(f"eps exponent {eps_exponent}, k {k}")

    # k = int(2 ** (k))

    final_test_acc, final_train_acc = gaussain_sampling_on_k(h_dis, y_data, reg_lambda, cls1_test_ts, cls2_test_ts, k)

    # print(exponent)
    # print(k)
    
    print("test acc", final_test_acc)
    print("train acc", final_train_acc)

    print("-" * 50)

    draw_test_acc_list.append(final_test_acc)
    draw_train_acc_list.append(final_train_acc)



--------------------------------------------------
eps exponent -0.4, k 14


100%|██████████| 10/10 [00:14<00:00,  1.46s/it]


test acc tensor(0.8025, device='cuda:0')
train acc tensor(0.8103, device='cuda:0')
--------------------------------------------------
--------------------------------------------------
eps exponent -0.3, k 22


100%|██████████| 10/10 [00:14<00:00,  1.49s/it]

test acc tensor(0.8630, device='cuda:0')
train acc tensor(0.8726, device='cuda:0')
--------------------------------------------------


In [22]:

def cal_k(eps, delta, beta):
    eta = 7e-3
    n = 1e3
    k_bound = (eps * eps * eta * eta) / (8 * math.log(1 / delta) * n * n * beta * beta)
    k = int(math.floor(k_bound))
    return k

beta = 1e-7
delta = 1e-3
# test_k = cal_k(1e-2, delta, beta)

# print(test_k)

# eps_exponent_list = [-1.0 + i * 0.2 for i in range(11)]
eps_exponent_list = [-0.5 + i * 0.1 for i in range(11)]

# eps_exponent_list = [-1.5 + i * 0.15 for i in range(11)]

for eps_exponent in eps_exponent_list:
    eps = 10 ** eps_exponent
    k = cal_k(eps, delta, beta)

    print(f"eps exponent {eps_exponent}, k {k}")

eps exponent -0.5, k 8
eps exponent -0.4, k 14
eps exponent -0.3, k 22
eps exponent -0.19999999999999996, k 35
eps exponent -0.09999999999999998, k 55
eps exponent 0.0, k 88
eps exponent 0.10000000000000009, k 140
eps exponent 0.20000000000000007, k 222
eps exponent 0.30000000000000004, k 352
eps exponent 0.4, k 559
eps exponent 0.5, k 886


In [28]:
draw_test_acc_list = [x.item() for x in draw_test_acc_list]
draw_train_acc_list = [x.item() for x in draw_train_acc_list]

In [29]:
# exponent_eps_list = [-3.0 + i * 0.2 for i in range(10)]
# draw_test_acc_list = []
# draw_train_acc_list = []

# print(k_list)
print(eps_exponent_list)
print(draw_test_acc_list)
print(draw_train_acc_list)

[-0.5, -0.4, -0.3, -0.19999999999999996, -0.09999999999999998, 0.0, 0.10000000000000009, 0.20000000000000007, 0.30000000000000004, 0.4, 0.5]
[0.7705000638961792, 0.8830000162124634, 0.8434999585151672, 0.9180000424385071, 0.9084998965263367, 0.9545000195503235, 0.9790000319480896, 0.9689998626708984, 0.9794999957084656, 0.9805000424385071, 0.981499969959259]
[0.7839500308036804, 0.8832500576972961, 0.8475000262260437, 0.9182001352310181, 0.9191500544548035, 0.9565500617027283, 0.9762499928474426, 0.9739500284194946, 0.9793000221252441, 0.9796000719070435, 0.9799000024795532]
